In [1]:
from requests import Request, Session
from selenium import webdriver 
from bs4 import BeautifulSoup
import re

In [2]:
#set the search term here
search = 'iphone'

In [3]:
page = 0

In [4]:
url = ['https://www.amazon.com.br/s?k=', search, '&page=', str(page)]

In [5]:
def urlfier(url):
    return ''.join(url)

urlfier(url)

'https://www.amazon.com.br/s?k=iphone&page=0'

In [49]:
#setting up the headers to allow multiple requests on API
headers = {
    
    #setting as default browser.
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36",
    
    #some configs i thought was important (i don't know yet)
    "authority": "www.amazon.com.br",
    "scheme": "https",
    "origin": "https://www.amazon.com.br",
    "referer": urlfier(url),
    
    "accept": "text/html,*/*",
    "accept-language": "en-US,en;q=0.9,ja-JP;q=0.8,ja;q=0.7,pt-BR;q=0.6,pt;q=0.5",
    "content-type": "application/json",
    "downlink": "50",
    "ect": "4g",
    "rtt": "100",
    "sec-ch-ua": "\"Google Chrome\";v=\"87\", \" Not;A Brand\";v=\"99\", \"Chromium\";v=\"87\"",
    "sec-ch-ua-mobile": "?0",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "x-amazon-rush-fingerprints": "AmazonRushAssetLoader:82DC4FCDBC1BD29C851447C5E56300ACEBC3484E|AmazonRushFramework:C63458EFD5C6F3B9DB4C55251BDF45CAB84ACA9E|AmazonRushRouter:C88B480F313C8089B887E17289DB6E9437386461|SearchAssets:68AEE251205924E94A40052B6F4BBA8540827F9F|DynamicImageLoader:72044AC19F5D25484C3A6A210715639EB294450A|SearchPartnerAssets:C3828FDE58240400D60660B659C2BB69C3AF2C8C",
    "x-amazon-s-fallback-url": "",
    "x-amazon-s-mismatch-behavior": "ALLOW",
    "x-amazon-s-swrs-version": "0265482AAA9506275F3AC0E0B1B2250D",
    "x-requested-with": "XMLHttpRequest"
}

In [50]:
payload = {
    "customer-action": "pagination",
    "page-content-type": "btf",
    "prefetch-type": "log",
}

In [51]:
# Prepared Session
URL = 'https://www.amazon.com.br/s?k=iphone&page=2&__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1607872074&ref=sr_pg_2'
s = Session()

prepped = Request('GET',
                  URL,
                  headers=headers,
                  params=payload
                 ).prepare()

In [52]:
# make the request
resp = s.send(prepped)
resp

<Response [200]>

In [53]:
# tratamento dos dados
content = resp.content


In [98]:
soup = BeautifulSoup(content, 'html.parser')

print(soup.prettify())

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="pt-br">
 <!-- sp:feature:head-start -->
 <head>
  <script>
   var aPageStart = (new Date()).getTime();
  </script>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   var ue_t0=ue_t0||+new Date();
  </script>
  <!-- sp:feature:cs-optimization -->
  <meta content="on" http-equiv="x-dns-prefetch-control"/>
  <link href="https://images-na.ssl-images-amazon.com" rel="dns-prefetch"/>
  <link href="https://m.media-amazon.com" rel="dns-prefetch"/>
  <link href="https://completion.amazon.com" rel="dns-prefetch"/>
  <script type="text/javascript">
   window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function

In [212]:
rep = []
ext = []
aux = []

#get all products of page
products = soup.select('div [data-index][data-uuid]')

#get name of product
name = extractText(products[0].select('h2 span')[0])
aux.append(name)

#get product price
price = products[8].select('span.a-offscreen')

#get stock of product
stock = products[10].select('div span[aria-label="Não disponível."] span[dir]')

if len(price) == 0:
    if len(stock) == 0:
        aux.append('Sem Estoque.')
    else:
        aux.append('Preço à tratar com vendedores.')
        

#get stars of product
stars = products[8].select('span.a-icon-alt')
if len(stars) == 0:
    stars = 'Sem avaliação'
else:
    stars = extractText(stars[0])

#rates in a product
rates = products[0].select('span[aria-label] a span.a-size-base')
if len(rates) == 0:
    rates = 'Sem avaliação'
else:
    rates = extractText(rates[0])

#link of an product
link = 'http://www.amazon.com.br' + products[0].select('h2 a')[0].get('href')

ext.append([link, stars, rates])

aux, ext

(['iPhone 8 Plus Apple 64GB Vermelho Tela Retina HD 5,5 IOS 11 4G e Camera de 12 MP',
  'Sem Estoque.'],
 [['http://www.amazon.com.br/iPhone-Apple-Vermelho-Retina-Camera/dp/B07CLNFJKN/ref=sr_1_49?__mk_pt_BR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&customer-action=pagination&dchild=1&keywords=iphone&page-content-type=btf&prefetch-type=log&qid=1607872074&sr=8-49',
   '5,0 de 5 estrelas',
   '26']])

In [176]:
def extractText(info):
    return BeautifulSoup(str(info), "html.parser").find(text=True)